In [1]:
import sys
import urllib.request
import socket
import socks
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import concurrent.futures
import requests
import warnings
import random
import json
warnings.filterwarnings("ignore")
#df = pd.read_csv('./est_price_final.csv')

In [2]:
import requests
from requests.structures import CaseInsensitiveDict
def send_req(postcode):
    url = "https://api-graphql-lambda.prod.zoopla.co.uk/graphql"

    headers = CaseInsensitiveDict()
    headers["user-agent"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36"
    headers["x-api-key"] = "3Vzj2wUfaP3euLsV4NV9h3UAVUR3BoWd5clv9Dvu"
    query = """query propertiesInPostcode {  properties(postcode: ) {    fullAddress    propertyId    __typename  }}"""
    query = query.replace("postcode: ","postcode: "+'"{}"'.format(postcode))
    #rint(query)
    resp = None
    try:
        resp = requests.post(url, headers=headers, json={'query': query})
    except requests.ConnectionError:
        print('Connection error for postcode: ',postcode)
    return resp


In [3]:
df = pd.DataFrame()

In [4]:
qr_cnt = 0
def collect_one(df, postcode):
    global qr_cnt
    qr_cnt+=1
    if qr_cnt%1000 == 0:
        print('num of queries: ', qr_cnt)
    response = send_req(postcode)
    if response == None:
        return [df,postcode]
    try:
        elevations = response.text
        data = json.loads(elevations)
        cnt = len(data['data']['properties'])
    except:
        return [df,postcode]
    #print(cnt)
    #print(data['data']['properties'][0]['fullAddress'])
    for x in range(cnt):
        df = df.append(pd.json_normalize(data['data']['properties'][x]))

    df = df.reset_index(drop=True)
    return [df,postcode]

In [5]:
base = pd.read_csv('ZP_RM_61k_with_address_base_estimated_misses.csv')

In [6]:
unique_postcodes = base['POSTCODE_LOCATOR'].unique()

In [7]:
base['POSTCODE_LOCATOR']

0         AL4 9AX
1         B17 9JX
2         B13 0BA
3         B13 0EP
4        AL10 9FP
           ...   
61118     LU1 3NY
61119     DH8 5TR
61120     TF2 8PY
61121    GU11 3LN
61122    YO12 6JN
Name: POSTCODE_LOCATOR, Length: 61123, dtype: object

In [8]:
len(unique_postcodes)

49435

In [9]:
unique_postcodes[1]

'B17 9JX'

In [10]:
for x in range(len(unique_postcodes)):
    cur = unique_postcodes[x]
    df = collect_one(df,cur)
    if x%10 == 0:
        print(x)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
num of queries:  1000
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180


In [50]:
collect_one(df,unique_postcodes[0])

[                                          fullAddress  propertyId  __typename
 0   2 Berrichon Crescent, Whitehouse, Milton Keyne...    32708947  Properties
 1   4 Berrichon Crescent, Whitehouse, Milton Keyne...    32615807  Properties
 2   6 Berrichon Crescent, Whitehouse, Milton Keyne...    32615808  Properties
 3   8 Berrichon Crescent, Whitehouse, Milton Keyne...    32601923  Properties
 4   10 Berrichon Crescent, Whitehouse, Milton Keyn...    32708948  Properties
 5   12 Berrichon Crescent, Whitehouse, Milton Keyn...    32593964  Properties
 6   14 Berrichon Crescent, Whitehouse, Milton Keyn...    32489215  Properties
 7   16 Berrichon Crescent, Whitehouse, Milton Keyn...    32489216  Properties
 8   18 Berrichon Crescent, Whitehouse, Milton Keyn...    32489217  Properties
 9   20 Berrichon Crescent, Whitehouse, Milton Keyn...    32489218  Properties
 10  22 Berrichon Crescent, Whitehouse, Milton Keyn...    32489219  Properties
 11  24 Berrichon Crescent, Whitehouse, Milton Keyn.

In [10]:
dff = pd.DataFrame()

In [11]:
dff

""


In [21]:
cur = 0
with concurrent.futures.ThreadPoolExecutor(max_workers=60) as executor:
    future_to_url = {executor.submit(collect_one,pd.DataFrame(),unique_postcodes[x]): x for x in range(10000,len(unique_postcodes))}
    for future in concurrent.futures.as_completed(future_to_url):
        ress = future.result()
        res = ress[0]
        res_pcode = ress[1]
        for i in range(len(res.index)):
            res.loc[i,'postcode'] = res_pcode
            
        dff = dff.append(res)
        cur+=1
        if cur%100 == 0:
            print(cur,len(dff.index))

100 314069
200 317088
300 320067
400 323346
500 326298
600 329763
700 332938
800 335875
num of queries:  11000
900 339413
1000 342910
1100 345611
1200 348940
1300 352423
1400 355862
1500 358618
1600 361459
1700 364422
1800 367362
1900 370436
num of queries:  12000
2000 373625
2100 376662
2200 379816
2300 383208
2400 386431
2500 389145
2600 392519
2700 395884
2800 399067
num of queries:  13000
2900 401932
3000 404981
3100 408200
3200 411364
3300 414660
3400 417704
3500 420535
3600 423767
3700 426924
3800 430206
3900 433297
num of queries:  14000
4000 436050
4100 439102
4200 442307
4300 445438
4400 448681
4500 451651
4600 455093
4700 458310
4800 461189
4900 464326
num of queries:  15000
5000 467507
5100 470388
5200 473371
5300 476454
5400 479715
5500 482358
5600 485508
5700 488734
num of queries:  16000
5800 491860
5900 494850
6000 498105
6100 500960
6200 504108
6300 507372
6400 510621
6500 513592
6600 516839
6700 519723
6800 522979
num of queries:  17000
6900 526021
7000 529346
7100 532

In [25]:
dff

,level_0,index,fullAddress,propertyId,__typename,postcode
0,0,0.0,"21 Hunters Way, Welwyn Garden City AL7 4NR",300935,Properties,AL7 4NR
1,1,1.0,"23 Hunters Way, Welwyn Garden City AL7 4NR",300936,Properties,AL7 4NR
2,2,2.0,"25 Hunters Way, Welwyn Garden City AL7 4NR",300937,Properties,AL7 4NR
3,3,3.0,"27 Hunters Way, Welwyn Garden City AL7 4NR",300938,Properties,AL7 4NR
4,4,4.0,"29 Hunters Way, Welwyn Garden City AL7 4NR",300939,Properties,AL7 4NR
...,...,...,...,...,...,...
1536202,50,NaN,"82 Hufling Lane, Burnley BB11 3QH",1451230,Properties,BB11 3QH
1536203,51,NaN,"83 Hufling Lane, Burnley BB11 3QH",30276375,Properties,BB11 3QH
1536204,52,NaN,"84 Hufling Lane, Burnley BB11 3QH",1451231,Properties,BB11 3QH
1536205,53,NaN,"86 Hufling Lane, Burnley BB11 3QH",1451232,Properties,BB11 3QH


In [181]:
cur

10000

In [20]:
len(dff['propertyId'].unique())

311048

In [24]:
dff.reset_index(inplace=True)

ValueError: cannot insert level_0, already exists

In [40]:
len(dff['propertyId'].unique())

63838

In [26]:
dff.to_csv('id_base_Nov29.csv')

In [223]:
#######################################################

In [184]:
df_p = pd.read_csv('C:/Users/hp-de/Downloads/England postcodes.csv')

In [185]:
df_p

,Postcode,In Use?,Latitude,Longitude,Easting,Northing,Grid Ref,County,District,Ward,...,Postcode area,Postcode district,Police force,Water company,Plus Code,Average Income,Sewage Company,Travel To Work Area,ITL level 2,ITL level 3
0,AL1 1AG,Yes,51.745290,-0.328628,515487.0,206498.0,TL154064,Hertfordshire,St Albans,Sopwell,...,AL,AL1,Hertfordshire,Affinity Water,9C3XPMWC+4G,52300.0,Thames Water,Luton,Bedfordshire and Hertfordshire,Hertfordshire CC
1,AL1 1AJ,Yes,51.744498,-0.328599,515491.0,206410.0,TL154064,Hertfordshire,St Albans,Sopwell,...,AL,AL1,Hertfordshire,Affinity Water,9C3XPMVC+QH,52300.0,Thames Water,Luton,Bedfordshire and Hertfordshire,Hertfordshire CC
2,AL1 1AR,Yes,51.739727,-0.317492,516270.0,205897.0,TL162058,Hertfordshire,St Albans,Cunningham,...,AL,AL1,Hertfordshire,Affinity Water,9C3XPMQM+V2,51500.0,Thames Water,Luton,Bedfordshire and Hertfordshire,Hertfordshire CC
3,AL1 1AS,Yes,51.749073,-0.335471,515005.0,206908.0,TL150069,Hertfordshire,St Albans,St Peters,...,AL,AL1,Hertfordshire,Affinity Water,9C3XPMX7+JR,65900.0,Thames Water,Luton,Bedfordshire and Hertfordshire,Hertfordshire CC
4,AL1 1AT,Yes,51.742011,-0.319421,516131.0,206148.0,TL161061,Hertfordshire,St Albans,Cunningham,...,AL,AL1,Hertfordshire,Affinity Water,9C3XPMRJ+R6,51500.0,Thames Water,Luton,Bedfordshire and Hertfordshire,Hertfordshire CC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1469188,YO90 1WR,Yes,53.959268,-1.087317,459983.0,451850.0,SE599518,North Yorkshire,York,Micklegate,...,YO,YO90,North Yorkshire,Yorkshire Water,9C5WXW57+P3,38300.0,NaN,York,North Yorkshire,York
1469189,YO90 1WT,Yes,53.960430,-1.092336,459652.0,451975.0,SE596519,North Yorkshire,York,Micklegate,...,YO,YO90,North Yorkshire,Yorkshire Water,9C5WXW65+53,38300.0,NaN,York,North Yorkshire,York
1469190,YO90 1WU,Yes,53.960430,-1.092336,459652.0,451975.0,SE596519,North Yorkshire,York,Micklegate,...,YO,YO90,North Yorkshire,Yorkshire Water,9C5WXW65+53,38300.0,NaN,York,North Yorkshire,York
1469191,YO90 1WW,Yes,53.960430,-1.092336,459652.0,451975.0,SE596519,North Yorkshire,York,Micklegate,...,YO,YO90,North Yorkshire,Yorkshire Water,9C5WXW65+53,38300.0,NaN,York,North Yorkshire,York


In [199]:
lsto = df_p['Postcode']
lsto = [x.split()[0] for x in lsto]

In [208]:
len(set(lsto))

2236

In [201]:
len(lsto)

1469193

In [166]:
collect_one(pd.DataFrame(),'NN12 7LD')

,fullAddress,propertyId,__typename
0,"1 Park Hall Cottages, Heathencote, Towcester N...",31781286,Properties
1,"2 Park Hall Cottages, Heathencote, Towcester N...",31785215,Properties
2,"Grooms Cottage, Heathencote, Towcester NN12 7LD",16538031,Properties
3,"1, Heathencote Cottage, Heathencote, Towcester...",16538037,Properties
4,"2, Heathencote Cottage, Heathencote, Towcester...",16538038,Properties
5,"Old Dairy Cottage, Heathencote, Towcester NN12...",28901679,Properties
6,"Park Hall Farm, Heathencote, Towcester NN12 7LD",16538032,Properties
7,"Stable House, Park Hall Farm, Heathencote, Tow...",16538033,Properties
8,"Park Hall Farm Cottage, Heathencote, Towcester...",16538034,Properties
9,"Park Hall House, Heathencote, Towcester NN12 7LD",16538035,Properties


In [220]:
from bs4 import BeautifulSoup
pattern = re.compile(r'[\"]\b(nsfw)\b\"[^\"]+[$,]')
 
page = requests.get("https://www.zoopla.co.uk/for-sale/property/al5/?added=24_hours")
soup = BeautifulSoup(page.content, 'html.parser')
 
data  = soup.find('script', type='application/json').string
b = re.search(pattern, data)
nsfw = b.group(0).replace(',','').split(':')[-1].capitalize()
print(nsfw)

AttributeError: 'NoneType' object has no attribute 'group'

In [224]:
my_json_string = soup.find('script', type='application/json').string

In [225]:
to_python = json.loads(my_json_string)

In [227]:
to_python['props']

{'pageProps': {'initialProps': {'analyticsTaxonomy': {'__typename': 'AnalyticsTaxonomy',
    'url': '/for-sale/property/al5/?added=24_hours',
    'areaName': 'AL5',
    'activity': 'listing_search',
    'brand': 'zoopla',
    'countryCode': 'gb',
    'countyAreaName': None,
    'currencyCode': 'GBP',
    'listingsCategory': 'residential',
    'outcode': 'AL5',
    'outcodes': ['AL5'],
    'page': '/for-sale/results/',
    'postalArea': 'AL',
    'radius': 5,
    'radiusAutoexpansion': 'true',
    'regionName': None,
    'resultsSort': 'newest_listings',
    'searchGuid': 'B1BAAAD0373411EC8FB9C747AC1A35BE',
    'searchIdentifier': '1e98faed1c597065b10a648a2cf99eef',
    'section': 'for-sale',
    'searchLocation': None,
    'viewType': 'list',
    'searchResultsCount': 3,
    'expandedResultsCount': 23,
    'totalResults': 26,
    'emailAllFormShown': False,
    'emailAllTotalAgents': 0,
    'bedsMax': None,
    'bedsMin': None,
    'priceMax': '',
    'priceMin': '',
    'location': No